# Load Demo with DAG for CIGNA

In [11]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,VSEKAR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [12]:
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta
import time
import re

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name

INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
# INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

# DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
# DATA_FILE = 'reduced_sample_data.json'
DATA_FILE = '2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz'

DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

SEGMENTS_PER_TASK = 15000

warehouses = config['SNOW_CONN']['warehouse']
warehouses = 'APP_WH,BANK1_WH,CLAKKAD_WH,DASTAN_BANK1_WH,DASTAN_BANK2_WH,DEV_PCTRANSPERANCY_DEMO_WH,HL7_WH'

# XSMALL | SMALL | MEDIUM | LARGE | XLARGE | XXLARGE | XXXLARGE | X4LARGE | X5LARGE | X6LARGE
warehouse_size = 'MEDIUM'


In [13]:
display(Markdown("### Cleanup block"))
# We will cleanup specific resources and artifacts from possible previous runs.

stmts = [
    f''' delete from segment_task_execution_status where data_file = '{DATA_FILE}'; '''
    ,f''' delete from task_to_segmentids where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_file_header where data_file = '{DATA_FILE}'; '''
    ,f''' delete from in_network_rates_segment_header where data_file = '{DATA_FILE}'; '''
    ,f''' alter stage {INPUT_DATA_STAGE} refresh; '''
]    
    
print(' truncating tables ...')
for stmt in stmts:
    sp_session.sql(stmt).collect()

print(f''' cleaning up files in external stage under path {TARGET_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
files = sp_session.sql(stmt).collect()
for r in files:
    stmt = f''' remove @{TARGET_DATA_STAGE}/{r['RELATIVE_PATH']}; '''
    sp_session.sql(stmt).collect()

### Cleanup block

 truncating tables ...
 cleaning up files in external stage under path raw_parsed/2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json/ ...


In [14]:
# reset the warehouse size to desired
for wh in warehouses.split(','):
    sp_session.sql(f''' alter warehouse {wh} set warehouse_size = {warehouse_size}; ''').collect()


---
## Data loading
We will be loading the segments and file header using DAG. 

In [15]:
# we build out the DAG
df = sp_session.call('in_network_rates_dagbuilder' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}' ,DATA_FILE 
    ,f"@{TARGET_DATA_STAGE}/{TARGET_FOLDER}" ,SEGMENTS_PER_TASK ,warehouses)

sp_session.sql(f''' alter stage {TARGET_DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

 Status of execution
{
  "data_file": "2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz",
  "root_task": "DAG_ROOT_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json",
  "status": true,
  "task_matrix_shape": [
    5,
    15
  ],
  "term_task": "TERM_tsk_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json"
}


The above operation results in defining the DAG in Snowflake like here. The task names are specific to the data file being parsed.
![](../../doc/soln_images/task_dags.png)

In [16]:
# Next we invoke the DAG

start_time = time.time()
print(f'Started at: {datetime.now().strftime("%H:%M:%S")}')

sql_stmts = [
    # default concurrency : 8
    f''' alter warehouse {config['SNOW_CONN']['warehouse']} set max_concurrency_level = 8 '''
    # XSMALL | SMALL | MEDIUM | LARGE | XLARGE | XXLARGE | XXXLARGE | X4LARGE | X5LARGE | X6LARGE
    ,f''' alter warehouse {config['SNOW_CONN']['warehouse']} set warehouse_size = MEDIUM; '''
    ,f''' execute task DAG_ROOT_{DATA_FILE_BASENAME_CLEANSED}; '''
]
for stmt in sql_stmts:
    print(stmt)
    sp_session.sql(stmt).collect()

end_time = time.time()
print(f'Ended at: {datetime.now().strftime("%H:%M:%S")}')

elapsed_time = end_time - start_time
elapsed = str(timedelta(seconds=elapsed_time))
print(f'Elapsed: {elapsed}')

Started at: 10:56:55
 alter warehouse DEMO_BUILD_WH set max_concurrency_level = 8 
 alter warehouse DEMO_BUILD_WH set warehouse_size = MEDIUM; 
 execute task DAG_ROOT_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json; 
Ended at: 10:56:56
Elapsed: 0:00:01.262966


---
## Inspection

In [17]:

print('Tasks to segments')
file_ingestion_df = sp_session.table('TASK_TO_SEGMENTIDS').filter(F.col('DATA_FILE') == F.lit(DATA_FILE)).to_pandas()
display(file_ingestion_df)

Tasks to segments


,BUCKET,DATA_FILE,ASSIGNED_TASK_NAME,FROM_IDX,TO_IDX,SEGMENTS_RECORD_COUNT,INSERTED_AT
0,38,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_570001_585000,570001,585000,14999,2023-01-09 07:56:37.442
1,12,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_180001_195000,180001,195000,14999,2023-01-09 07:56:37.442
2,16,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_240001_255000,240001,255000,14999,2023-01-09 07:56:37.442
3,42,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_630001_645000,630001,645000,14999,2023-01-09 07:56:37.442
4,68,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_1020001_1035000,1020001,1035000,14999,2023-01-09 07:56:37.442
...,...,...,...,...,...,...,...
70,51,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_765001_780000,765001,780000,14999,2023-01-09 07:56:37.442
71,49,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_735001_750000,735001,750000,14999,2023-01-09 07:56:37.442
72,37,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_555001_570000,555001,570000,14999,2023-01-09 07:56:37.442
73,36,2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz,T_2022_12_01_cigna_health_life_insurance_company_national_ppo_in_network_rates_json_540001_555000,540001,555000,14999,2023-01-09 07:56:37.442


In [18]:

print('Tasks ,warehouses and state')
sp_session.sql(f''' SHOW TASKS IN  DATABASE {config['APP_DB']['database']}; ''').collect()
stmt = f'''
    select "name" as task_name
        ,"warehouse" as warehouse
        ,"state" as state
    from table(result_scan(last_query_id()))
    where "name" like '%{DATA_FILE_BASENAME_CLEANSED.upper()}%'
       -- and state != 'suspended'
    order by state;
'''
df = sp_session.sql(stmt).to_pandas()
display(df)

Tasks ,warehouses and state


,TASK_NAME,WAREHOUSE,STATE
0,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_1110001_999999999,DEV_PCTRANSPERANCY_DEMO_WH,started
1,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_0_15000,BANK1_WH,started
2,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_1005001_1020000,CLAKKAD_WH,started
3,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_1020001_1035000,DASTAN_BANK1_WH,started
4,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_1035001_1050000,DASTAN_BANK2_WH,started
...,...,...,...
73,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_975001_990000,APP_WH,started
74,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_990001_1005000,BANK1_WH,started
75,T_FH_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON,APP_WH,started
76,T_2022_12_01_CIGNA_HEALTH_LIFE_INSURANCE_COMPANY_NATIONAL_PPO_IN_NETWORK_RATES_JSON_885001_900000,CLAKKAD_WH,started


--- 
### Closeout

    With that we are finished this section of the demo setup

In [19]:
# sp_session.close()
print('Finished!!!')

Finished!!!
